In [1]:
import sys
import os
from tempfile import mkdtemp
sys.path.insert(0, '..')

import numpy as np
from process_bigraph import Composite, pp
from process_bigraph.experiments.parameter_scan import RunProcess

from biosimulator_processes import CORE 


Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Available processes:
[ 'console-emitter',
  'ram-emitter',
  'composite',
  'cobra-process',
  'copasi-process',
  'tellurium-process',
  'utc-amici',
  'utc-copasi',
  'utc-tellurium',
  'copasi-step',
  'tellurium-step',
  'amici-step',
  'plotter',
  'plotter2d',
  'utc-comparator']


1. Define inputs

In [2]:
omex_dir = '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator'
omex_fp = omex_dir + '.omex'
sbml_fp = os.path.join(omex_dir, 'BIOMD0000000012_url.xml')
sedml_fp = os.path.join(omex_dir, 'simulation.sedml')
reports_path = os.path.join(omex_dir, 'reports.h5')

2. Define composition document (manuscript?)

In [3]:
manuscript = {
    'amici': {
        '_type': 'step',
        'address': 'local:utc-amici',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store'],
            'model_parameters': ['amici_model_parameters_store'],
            'reactions': ['amici_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store']
        }
    },
    'copasi': {
        '_type': 'step',
        'address': 'local:utc-copasi',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store'],
            'model_parameters': ['copasi_model_parameters_store'],
            'reactions': ['copasi_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store']
        }
    },
    'tellurium': {
        '_type': 'step',
        'address': 'local:utc-tellurium',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store'],
            'model_parameters': ['tellurium_model_parameters_store'],
            'reactions': ['tellurium_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store']
        }
    },
    'comparison': {
        '_type': 'step',
        'address': 'local:utc-comparator',
        'config': {
            'simulators': ['amici', 'copasi', 'tellurium'],
        },
        'inputs': {
            'time': ['time_store'],
            'amici_floating_species': ['amici_floating_species_store'],
            'copasi_floating_species': ['copasi_floating_species_store'],
            'tellurium_floating_species': ['tellurium_floating_species_store'],
        },
        'outputs': {
            'results': ['results_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'results': 'tree[string]',
            }
        },
        'inputs': {
            'results': ['results_store'],
        }
    }
}


output_bridge = {
    'inputs': {
        'results': ['results_store']
    },
    'outputs': {
        'results': ['results_store']
    }
}

3. Create composition with doc specifications

In [4]:
composition_config = {'state': manuscript, 'bridge': output_bridge}
composition = Composite(
    config=composition_config,
    core=CORE
)

2024-06-05 16:45:42.867 - amici.sbml_import - INFO - Finished importing SBML                         (4.65E-02s)
2024-06-05 16:45:42.920 - amici.sbml_import - INFO - Finished processing SBML observables            (5.01E-02s)
2024-06-05 16:45:42.926 - amici.sbml_import - INFO - Finished processing SBML event observables      (5.00E-07s)
2024-06-05 16:45:42.951 - amici.de_model - INFO - Finished computing xdot                            (3.47E-03s)
2024-06-05 16:45:42.959 - amici.de_model - INFO - Finished computing x0                              (3.11E-03s)
2024-06-05 16:45:42.980 - amici.de_model - INFO - Finished computing w                               (1.70E-02s)
2024-06-05 16:45:43.792 - amici.de_export - INFO - Finished generating cpp code                      (7.98E-01s)
2024-06-05 16:45:52.327 - amici.de_export - INFO - Finished compiling cpp code                       (8.53E+00s)


In [5]:
dir(composition)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_updates',
 'bridge',
 'bridge_updates',
 'composition',
 'config',
 'config_schema',
 'core',
 'cycle_step_state',
 'determine_steps',
 'edge_paths',
 'emit_port',
 'emitter_paths',
 'expire_process_paths',
 'find_instance_paths',
 'front',
 'gather_results',
 'global_time_precision',
 'initial_state',
 'inputs',
 'interface',
 'invoke',
 'merge',
 'node_dependencies',
 'outputs',
 'process_paths',
 'process_schema',
 'process_update',
 'reset_step_state',
 'run',
 'run_process',
 'run_steps',
 'state',
 'step_dependencies',
 'step_paths',
 'step_triggers',
 'steps_remaining',
 'steps_run',
 'to_ru

In [8]:
pp(composition.state)

{ 'amici': { '_inputs': { 'floating_species': 'tree[string]',
                          'model_parameters': { 'KM': { '_apply': 'set',
                                                        '_type': 'float'},
                                                'a0_tr': { '_apply': 'set',
                                                           '_type': 'float'},
                                                'a_tr': { '_apply': 'set',
                                                          '_type': 'float'},
                                                'alpha': { '_apply': 'set',
                                                           '_type': 'float'},
                                                'alpha0': { '_apply': 'set',
                                                            '_type': 'float'},
                                                'average mRNA life time': { '_apply': 'set',
                                                                            '_typ